In [ ]:
!pip install -qU "setfit[absa]"
!pip install -q contractions
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from datasets import load_dataset
from setfit import AbsaModel
import contractions
import nltk
import spacy
import re

c:\Users\nghialt\.conda\envs\ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Tải về các gói cần thiết từ NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
# Tải mô hình ngôn ngữ tiếng Anh
nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nghialt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nghialt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
def processing_text(text, lower_case=True):
    # Preprocess text: lowercasing, contraction expansion, and tokenization
    lower_text = text.lower() if lower_case else text
    expanded_text = contractions.fix(lower_text)
    processed_text = re.sub(r'[^a-zA-Z0-9\s\']', ' ', expanded_text)
    tokens = nltk.word_tokenize(processed_text)
    return ' '.join(tokens)

In [5]:
# Create a model with a chosen sentence transformer from the Hub
model = AbsaModel.from_pretrained(
    "tomaarsen/setfit-absa-paraphrase-mpnet-base-v2-restaurants-aspect",
    "tomaarsen/setfit-absa-paraphrase-mpnet-base-v2-restaurants-polarity"
)

c:\Users\nghialt\.conda\envs\ai\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nghialt\.cache\huggingface\hub\models--tomaarsen--setfit-absa-paraphrase-mpnet-base-v2-restaurants-aspect. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\nghialt\.conda\envs\ai\lib\site-packages\sklearn\base.py:376

In [6]:
def detect_span(sentence, query_span):
    """
    Detect the position (start, end) of a query span in a given sentence.
    """
    # Process the sentence using spaCy
    doc = nlp(sentence)

    # Split the query span into tokens
    query_tokens = query_span.split()

    span_position = None
    # Find the position of the span
    for token in doc:
        # Check if the token matches the first token of the query span
        if token.text == query_tokens[0]:
            # Attempt to match the entire query span
            span = doc[token.i:token.i + len(query_tokens)]
            if span.text == query_span:
                span_position = (token.i, token.i + len(query_tokens) - 1)  # Inclusive indexing
                break
    return span_position

def predict_sentence(sentence):
    """
    Predict aspects and polarities for a given sentence and label tokens.
    """
    # Preprocess the sentence
    processed_sentence = processing_text(sentence)
    # processed_sentence = sentence
    # Get predictions from the model
    pred = model.predict([processed_sentence])[0]

    positions, polarities = [], []
    for aspect in pred:
        # Detect span positions for each aspect term
        span_position = detect_span(processed_sentence, aspect['span'])
        if span_position:
            positions.append(span_position)
            # Map polarity to a label
            if aspect['polarity'] == 'positive':
                polarities.append('POS')
            elif aspect['polarity'] == 'negative':
                polarities.append('NEG')
            elif aspect['polarity'] == 'neutral':
                polarities.append('NEU')

    prev_end = 0

    # Tokenize the processed sentence
    tokens = nltk.word_tokenize(processed_sentence)
    labels = []

    # Assign labels to tokens based on aspect spans and polarities
    for (start, end), polarity in zip(positions, polarities):
        labels.extend(['O'] * (start - prev_end))
        if start == end:
            labels.append(f'S-{polarity}')
        else:
            labels.extend([f'B-{polarity}'] + ['I-' + polarity] * (end - start - 1) + [f'E-{polarity}'])
        prev_end = end + 1

    # Add 'O' labels for tokens outside any aspect span
    labels.extend(['O'] * (len(tokens) - prev_end))

    return list(zip(tokens, labels))

def predict_paragraph(paragraph):
    """
    Predict aspects and polarities for an entire paragraph.
    """
    # Split the paragraph into sentences using spaCy
    doc = nlp(paragraph)
    sentences = [sent.text for sent in doc.sents]
    results = []
    for sentence in sentences:
        results.append(predict_sentence(sentence))

    return results

In [9]:
sentence = "As I waited in line. A worker in a black polo shirt (supervisor I think?) picked his nose, pushed down the garbage, picked up a piece of trash, and fixed his pants with the same hand he had a glove on, then went back to serving food. I asked the girl helping me if he could change his gloves please as he was about to touch my plate, she told him, and he just walked around until I left.. with the same glove on… She finished putting together my plate. He never changed his glove as far as I know. Terrible food safety."
predict_paragraph(sentence)

[[('as', 'O'), ('i', 'O'), ('waited', 'O'), ('in', 'O'), ('line', 'S-NEU')],
 [('a', 'O'),
  ('worker', 'S-NEU'),
  ('in', 'O'),
  ('a', 'O'),
  ('black', 'O'),
  ('polo', 'B-NEU'),
  ('shirt', 'I-NEU'),
  ('supervisor', 'E-NEU'),
  ('i', 'O'),
  ('think', 'O'),
  ('picked', 'O'),
  ('his', 'O'),
  ('nose', 'O'),
  ('pushed', 'O'),
  ('down', 'O'),
  ('the', 'O'),
  ('garbage', 'S-NEU'),
  ('picked', 'O'),
  ('up', 'O'),
  ('a', 'O'),
  ('piece', 'O'),
  ('of', 'O'),
  ('trash', 'S-NEU'),
  ('and', 'O'),
  ('fixed', 'O'),
  ('his', 'O'),
  ('pants', 'O'),
  ('with', 'O'),
  ('the', 'O'),
  ('same', 'O'),
  ('hand', 'O'),
  ('he', 'O'),
  ('had', 'O'),
  ('a', 'O'),
  ('glove', 'O'),
  ('on', 'O'),
  ('then', 'O'),
  ('went', 'O'),
  ('back', 'O'),
  ('to', 'O'),
  ('serving', 'O'),
  ('food', 'S-NEU')],
 [('i', 'O'),
  ('asked', 'O'),
  ('the', 'O'),
  ('girl', 'O'),
  ('helping', 'O'),
  ('me', 'O'),
  ('if', 'O'),
  ('he', 'O'),
  ('could', 'O'),
  ('change', 'O'),
  ('his', 'O'),
  